In [14]:
import sys
import subprocess

def install_package(package):
    if isinstance(package, str):
        package = package.split(" ")
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + package)

In [15]:
install_package("trulens trulens-providers-openai trulens-apps-llamaindex llama_index openai")

  Using cached trulens-1.2.6-py3-none-any.whl.metadata (4.0 kB)
  Using cached trulens_providers_openai-1.2.6-py3-none-any.whl.metadata (1.1 kB)
  Using cached trulens_apps_llamaindex-1.2.6-py3-none-any.whl.metadata (1.1 kB)
  Using cached trulens_core-1.2.6-py3-none-any.whl.metadata (1.3 kB)
  Using cached trulens_dashboard-1.2.6-py3-none-any.whl.metadata (1.6 kB)
  Using cached trulens_feedback-1.2.6-py3-none-any.whl.metadata (16 kB)
  Using cached trulens_eval-1.2.6-py3-none-any.whl.metadata (1000 bytes)
  Using cached trulens_apps_langchain-1.2.6-py3-none-any.whl.metadata (1.0 kB)
INFO: pip is looking at multiple versions of trulens-apps-langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached trulens_apps_langchain-1.2.5-py3-none-any.whl.metadata (1.0 kB)
  Using cached trulens_apps_langchain-1.2.4-py3-none-any.whl.metadata (1.0 kB)
  Using cached trulens_apps_langchain-1.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cach

  Using cached streamlit-1.40.1-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached streamlit_aggrid-1.0.5-py3-none-any.whl.metadata (8.2 kB)
  Using cached streamlit_extras-0.5.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached streamlit_pills-0.3.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached Mako-1.3.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
  Using cached notebook-7.2.2-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached altair-5.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached protobuf-5.28.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using ca

In [41]:
import json
import pandas as pd

with open("../dataset.json") as f:
    data = json.load(f)

# questions = data['question']
# ground_truths = data['ground_truths']
# answers = data['answer']
# contexts = data['contexts']

dataset = {
    "query": data['question'],
    "response": data['answer'],
    "contexts": data['contexts'],
}
df = pd.DataFrame(dataset)
df.head()

,query,response,contexts
0,What is ECMWF’s main focus in weather prediction?,ECMWF's main focus in weather prediction is on...,[Seasonal Forecasting at ECMWF | CMEMS\n\r\nSk...
1,How does ECMWF use satellite data to improve w...,ECMWF uses satellite data to improve weather f...,[Seasonal Forecasting at ECMWF | CMEMS\n\r\nSk...
2,What is the Integrated Forecasting System (IFS...,The Integrated Forecasting System (IFS) is a g...,"[In May 2015, a lake model was implemented in ..."
3,"How does ECMWF’s seasonal forecasting work, an...",ECMWF's seasonal forecasting works by combinin...,[Seasonal Forecasting at ECMWF | CMEMS\n\r\nSk...
4,What are large-scale vs. convective precipitat...,"In ECMWF's models, large-scale precipitation a...","[```{r, eval=FALSE} client$datasets(""EO:ECMWF:..."


In [18]:
from trulens.apps.virtual import VirtualApp

virtual_app = VirtualApp()

In [24]:
import os
TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')

assert TOGETHER_API_KEY

In [32]:
# from llama_index.llms.openai import OpenAI
# from llama_index.llms.together import TogetherLLM

# llm_model_params = {
#     "api_key": TOGETHER_API_KEY,
#     "base_url": "https://api.together.xyz/v1",
#     "model": "Qwen/Qwen2.5-7B-Instruct-Turbo"
# }

# provider = TogetherLLM(**llm_model_params)
from trulens.providers.openai import OpenAI
llm_model_params = {
    "api_key": TOGETHER_API_KEY,
    "base_url": "https://api.together.xyz/v1",
    "model": "Qwen/Qwen2.5-7B-Instruct-Turbo"
}
provider = OpenAI(model_engine=llm_model_params.pop('model'), **llm_model_params)

In [34]:
from trulens.core import Feedback

context = VirtualApp.select_context()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retriever.get_context.rets[:] .
✅ In Groundedness, input source will be set to __record__.app.retriever.get_context.rets[:].collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [35]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession()
run_dashboard(session)

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.100.189:44661 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [45]:
from trulens.apps.virtual import TruVirtual

virtual_recorder = TruVirtual(
    app_name="RAG",
    app_version="simple-v2",
    app=virtual_app,
    feedbacks=[f_context_relevance, f_groundedness, f_qa_relevance],
)

In [46]:
virtual_records = virtual_recorder.add_dataframe(df)

[nltk_data] [Errno 2] No such file or directory:
[nltk_data]     '/home/tibi/nltk_data/tokenizers/punkt_tab.zip'
[nltk_data] [Errno 2] No such file or directory:
[nltk_data]     '/home/tibi/nltk_data/tokenizers/punkt_tab.zip'
[nltk_data] [Errno 2] No such file or directory:
[nltk_data]     '/home/tibi/nltk_data/tokenizers/punkt_tab.zip'
[nltk_data] [Errno 2] No such file or directory:
[nltk_data]     '/home/tibi/nltk_data/tokenizers/punkt_tab.zip'
[nltk_data] Error with downloaded zip file
[nltk_data] Error with downloaded zip file
[nltk_data] [Errno 2] No such file or directory:
[nltk_data]     '/home/tibi/nltk_data/tokenizers/punkt_tab.zip'
/mnt/docker/work/sd/notebooks/.venv/lib/python3.11/site-packages/trulens/core/feedback/feedback.py:974: UserWarning: Feedback function Context Relevance with aggregation <function mean at 0x7d8658392200> had no inputs.
  warnings.warn(
[nltk_data] [Errno 2] No such file or directory:
[nltk_data]     '/home/tibi/nltk_data/tokenizers/punkt_tab.zip'
